# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielchang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/danielchang/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielchang/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# import statements
import re
import numpy as np
import pandas as pd
import warnings
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine
import pickle

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier

warnings.simplefilter('ignore')

In [3]:
# load data from database
engine = create_engine('sqlite:///ETLPipelinePrep.db')
df = pd.read_sql_table('Disasters_ETL', engine)
X = df.message
y = df[df.columns[4:]]

In [10]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # normalization word tokens and remove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    normalized = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return normalized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x125ba3430>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
y_pred=pipeline.predict(X_test)
category_names = y.columns

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i])) #[:,i] is aimed at the specific column and all rows

related :
              precision    recall  f1-score   support

           0       0.67      0.41      0.51      1485
           1       0.84      0.94      0.89      5020
           2       0.74      0.29      0.41        49

    accuracy                           0.82      6554
   macro avg       0.75      0.55      0.60      6554
weighted avg       0.80      0.82      0.80      6554

request :
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5412
           1       0.85      0.49      0.62      1142

    accuracy                           0.90      6554
   macro avg       0.88      0.74      0.78      6554
weighted avg       0.89      0.90      0.88      6554

offer :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50   

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6476
           1       0.00      0.00      0.00        78

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

other_infrastructure :
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6260
           1       0.00      0.00      0.00       294

    accuracy                           0.95      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.91      0.95      0.93      6554

weather_related :
              precision    recall  f1-score   support

           0       0.90      0.95      0.93      4776
           1       0.86      0.73      0.79      1778

    accuracy                           0.89      6554
   macro avg       0.88      0.84      0.86      6554
weighted avg       0.89      0.89

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x125ba3430>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x125ba3430>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_i

In [19]:
# GridSearchCV help: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
parameters = {
    #'vect__ngram_range': ((1, 1),(1,2)), 
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_split': [2, 3, 4],
    
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)

In [20]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50
[CV 1/5; 1/6] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50;, score=0.258 total time= 2.1min
[CV 2/5; 1/6] START clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50
[CV 2/5; 1/6] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50;, score=0.266 total time= 2.1min
[CV 3/5; 1/6] START clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50
[CV 3/5; 1/6] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50;, score=0.261 total time= 2.0min
[CV 4/5; 1/6] START clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50
[CV 4/5; 1/6] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50;, score=0.253 total time= 1.9min
[CV 5/5; 1/6] START clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50
[CV 5/5; 1/6] E

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x125ba3430>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [2, 3, 4],
                         'clf__estimator__n_estimators': [50, 100]},
             verbose=10)

In [21]:
#Youtube help - https://www.youtube.com/watch?v=TvB_3jVIHhg
print(cv.best_params_)

{'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 100}


In [22]:
print(cv.best_estimator_)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x125ba3430>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=3)))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
# built a new model, using best_estimator_
# https://stackoverflow.com/questions/45074698/how-to-pass-elegantly-sklearns-gridseachcvs-best-parameters-to-another-model
tuned_model = cv.best_estimator_
tuned_pred = tuned_model.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], tuned_pred[:,i])) 

related :
              precision    recall  f1-score   support

           0       0.68      0.40      0.50      1485
           1       0.84      0.94      0.89      5020
           2       0.67      0.29      0.40        49

    accuracy                           0.81      6554
   macro avg       0.73      0.54      0.60      6554
weighted avg       0.80      0.81      0.80      6554

request :
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5412
           1       0.84      0.48      0.61      1142

    accuracy                           0.89      6554
   macro avg       0.87      0.73      0.77      6554
weighted avg       0.89      0.89      0.88      6554

offer :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50   

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6260
           1       0.00      0.00      0.00       294

    accuracy                           0.95      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.91      0.95      0.93      6554

weather_related :
              precision    recall  f1-score   support

           0       0.90      0.95      0.93      4776
           1       0.85      0.73      0.78      1778

    accuracy                           0.89      6554
   macro avg       0.88      0.84      0.86      6554
weighted avg       0.89      0.89      0.89      6554

floods :
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6058
           1       0.90      0.44      0.59       496

    accuracy                           0.95      6554
   macro avg       0.93      0.72      0.78      6554
weighted avg       0.95      0.95      0.95    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
# AdaBoost Classifer help: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
new_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators = 100)))
    ])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
new_pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x125ba3430>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(n_estimators=100)))])

In [27]:
y_pred_new = new_pipeline.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred_new[:,i]))

related :
              precision    recall  f1-score   support

           0       0.61      0.16      0.25      1506
           1       0.79      0.97      0.87      4994
           2       0.36      0.17      0.23        54

    accuracy                           0.78      6554
   macro avg       0.59      0.43      0.45      6554
weighted avg       0.74      0.78      0.72      6554

request :
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      5442
           1       0.75      0.55      0.63      1112

    accuracy                           0.89      6554
   macro avg       0.83      0.75      0.79      6554
weighted avg       0.89      0.89      0.89      6554

offer :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50   

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6458
           1       0.25      0.08      0.12        96

    accuracy                           0.98      6554
   macro avg       0.62      0.54      0.56      6554
weighted avg       0.98      0.98      0.98      6554

other_infrastructure :
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      6281
           1       0.28      0.10      0.15       273

    accuracy                           0.95      6554
   macro avg       0.62      0.54      0.56      6554
weighted avg       0.93      0.95      0.94      6554

weather_related :
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      4714
           1       0.84      0.69      0.76      1840

    accuracy                           0.88      6554
   macro avg       0.87      0.82      0.84      6554
weighted avg       0.87      0.88

### 9. Export your model as a pickle file

In [28]:
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
#https://stackoverflow.com/questions/34143829/sklearn-how-to-save-a-model-created-from-a-pipeline-and-gridsearchcv-using-jobli
#pickling the best model and saving data(I believe tuned_model is the best one.) 
pickle.dump(tuned_model, open('disaster_model.sav', 'wb'))

# loading data
loaded_model = pickle.load(open('disaster_model.sav', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.797985962770827


In [29]:
# out of curiosity, I wanted to check out the result for the new model as well
pickle.dump(new_pipeline, open('new_disaster_model.sav', 'wb'))

# loading data
loaded_model = pickle.load(open('new_disaster_model.sav', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.22566371681415928


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.